In [62]:
import pandas as pd
from tqdm import tqdm
from experiments import Experiment

In [63]:
df = pd.read_csv('../data/german_credit.csv')

In [64]:
features = [
'status_checking_account',
'duration_in_month',
'credit_history',
'purpose',
'savings',
'employement_since',
'installment_rate',
'debters',
'resident_since',
'property',
'age',
'other_installments',
'housing',
'num_credits',
'job',
'num_liable',
'telephone',
'foreign_worker'
]
protected_attribute = "gender"
task_types =  ["is_good_loan", "is_high_credit"]
model_types = ["logistic", "gbm", "nn", "svm", "tree"]
n_train = 800
n_test = 200

# Randomness Based on Distance to Threshold

In [65]:
random_thresholds = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
data_scales = [10, 9, 8, 7, 6, 5, 4, 3, 2]
random_seeds = list(range(5))

results = []

for data_scale in tqdm(data_scales):
    for random_seed in tqdm(random_seeds):
        exp = Experiment(df, features, protected_attribute, task_types,
                             model_types, random_thresholds, n_train, n_test, random_seed, data_scale)

        exp.pretrain_models()
        results.append(exp.experiment_baseline())
        results.append(exp.experiment_tasks())
        results.append(exp.experiment_models())
        
        results.append(exp.experiment_partitions(min(5, data_scale)))
results = pd.concat(results)
results.to_csv("german_credit_results.csv", index=False)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree



 20%|█████████                                    | 1/5 [00:04<00:19,  4.86s/it]

is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm



 40%|██████████████████                           | 2/5 [00:09<00:14,  4.76s/it]

...

is_high_credit tree
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm



 80%|████████████████████████████████████         | 4/5 [00:18<00:04,  4.53s/it]

is_high_credit tree
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm



100%|█████████████████████████████████████████████| 9/9 [03:47<00:00, 25.30s/it]

is_high_credit tree


# Randomness Based on Conformal Prediction

In [66]:
random_thresholds = [0, 0.05, 0.1, 0.2, 0.3, 0.4]
data_scales = [10, 9, 8, 7, 6, 5, 4, 3, 2]
random_seeds = [0,3,4,5,6]

results = []
conformal_pred = True

for data_scale in tqdm(data_scales):
    for random_seed in tqdm(random_seeds):
        exp = Experiment(df, features, protected_attribute, task_types,
                             model_types, random_thresholds, n_train, n_test, random_seed, data_scale, conformal_pred)

        exp.pretrain_models()
        results.append(exp.experiment_baseline())
        results.append(exp.experiment_tasks())
        results.append(exp.experiment_models())
        
        results.append(exp.experiment_partitions(min(5, data_scale)))
results = pd.concat(results)
results.to_csv("german_credit_results_conformal.csv", index=False)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree



 20%|█████████                                    | 1/5 [00:06<00:25,  6.45s/it]

is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree



 40%|██████████████████                           | 2/5 [00:12<00:19,  6.42s/it]

...

is_high_credit tree
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm



 80%|████████████████████████████████████         | 4/5 [00:23<00:05,  5.85s/it]

is_high_credit tree
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm



100%|█████████████████████████████████████████████| 9/9 [04:49<00:00, 32.17s/it]

is_high_credit tree
